### Stress testing for historical regimes 

In [12]:
%pip install ib-insync

Note: you may need to restart the kernel to use updated packages.


#### Test connection: 

In [13]:
from ib_insync import IB
ib = IB()

In [14]:
await ib.connectAsync('127.0.0.1', 7497, clientId=9)

<IB connected to 127.0.0.1:7497 clientId=9>

In [15]:
ib.isConnected()

True

In [16]:
from ib_insync import Stock

spy = Stock('SPY', 'ARCA', 'USD')
qualified = await ib.qualifyContractsAsync(spy)
qualified

### note: IBKR requires contracts to be fully qualified before use. 
### The qualifyContractsAsync call resolves ambiguous ticker information and assigns a unique contract ID, 
### ensuring that historical data requests and trades are routed to the correct instrument. 

[Stock(conId=756733, symbol='SPY', exchange='ARCA', primaryExchange='ARCA', currency='USD', localSymbol='SPY', tradingClass='SPY')]

In [17]:
import os
import pandas as pd
from ib_insync import IB, Stock, Forex, util

In [18]:
### Load portfolio.csv 
PORTFOLIO_PATH = "portfolio.csv"

In [19]:
portfolio = pd.read_csv(PORTFOLIO_PATH)
portfolio.columns = [c.strip() for c in portfolio.columns]  
portfolio

,symbol,asset_class,exchange,currency,weight
0,SPY,ETF,ARCA,USD,0.4
1,TLT,ETF,ARCA,USD,0.3
2,GLD,ETF,ARCA,USD,0.1
3,EURUSD,FX,IDEALPRO,USD,0.2


Justification of asset is sample portfolio: 

The portfolio is constructed to represent a simplified yet diversified multi-asset allocation that captures the main macroeconomic risk factors.

Equity market exposure (SPY) serves as the primary source of growth and systemic risk. Due to it being highly sensitive to recessions, liquidity shocks and financial crises, it is expected to suffer most during crises and drive portfolio drawdowns. 

Long-duration government bonds (TLT) are included to test their role as a defensive hedge during market crises because it is sensitive to interest rate cuts and flight-to-safety episodes. It is expected to do well during equity crashes and hedge equity risk. 

Gold (GLD) provides exposure to real assets and tail-risk protection. It is sensitive to crisis uncertainty, monetary debasement and inflation expectations. 

EURUSD captures currency and global macro transmission effects. It is sensitive to risk-on/ risk off dynamics and US vs Europe divergene. 

In [20]:
### Contract builder to construct appropriate IBKR contract object for each asset based on its asset class

def make_contract(row: pd.Series):
    symbol = str(row["symbol"]).strip()
    asset_class = str(row["asset_class"]).strip().upper()
    exchange = str(row.get("exchange", "")).strip()
    currency = str(row.get("currency", "USD")).strip()

    if asset_class in {"FX", "FOREX"}:
        # e.g. "EURUSD"
        return Forex(symbol)
    else:
        # ETFs/stocks (SPY, TLT, GLD)
        # For US ETFs, exchange ARCA is fine.
        return Stock(symbol, exchange, currency)

In [21]:
### Robust daily history puller (chunks by 1Y to avoid IBKR limits) 

### This function retrieves historical daily price data for a given asset from IBKR by splitting the request into one-year chunks to avoid IBKR’s historical data limits. 
### Starting from the specified end date, it repeatedly requests one year of daily bars, converts each response into a DataFrame, and moves backwards in time until the start date is reached or no more data is available. 
### The individual chunks are then combined, de-duplicated, sorted chronologically, and trimmed to the exact date range requested. 

async def fetch_daily_history(
    ib: IB,
    contract,
    start_date: str,
    end_date: str,
    whatToShow: str,
    useRTH: bool,
    chunk: str = "1 Y",
    barSizeSetting: str = "1 day",
):
    """
    Pull daily bars between [start_date, end_date] inclusive.
    Uses chunking (1 year per request) and stitches results together.
    """
    start = pd.Timestamp(start_date)
    end = pd.Timestamp(end_date)

    all_chunks = []
    end_dt = end

    # Loop backwards from end_date in chunks until we cover start_date
    while end_dt >= start:
        bars = await ib.reqHistoricalDataAsync(
            contract,
            endDateTime=end_dt.strftime("%Y%m%d 23:59:59"),
            durationStr=chunk,
            barSizeSetting=barSizeSetting,
            whatToShow=whatToShow,
            useRTH=useRTH,
            formatDate=1,
        )

        if not bars:
            break

        df = util.df(bars)
        
        if df is None or df.empty:
            break

        df["date"] = pd.to_datetime(df["date"])
        all_chunks.append(df)

        # Move the end pointer to just before the earliest bar we received
        earliest = df["date"].min()
        end_dt = earliest - pd.Timedelta(days=1)

    if not all_chunks:
        return pd.DataFrame()

    out = pd.concat(all_chunks, ignore_index=True)
    out = out.drop_duplicates(subset=["date"]).sort_values("date")
    out = out[(out["date"] >= start) & (out["date"] <= end)].reset_index(drop=True)
    return out

In [22]:
### qualify + pull + save for each asset 

### This function converts each portfolio entry into an IBKR contract, qualifies those contracts to ensure they are uniquely identified and usable by the API, 
### and then iterates through each asset to pull its daily historical data over the specified date range. Depending on the asset class, it requests the appropriate data type 
### (midpoint prices for FX and trade prices for equities/ETFs) and uses the robust chunked history function to handle IBKR data limits. 
### The resulting price series for each asset is stored in a dictionary and saved as a CSV file. 

async def pull_all_assets_history(
    ib: IB,
    portfolio_df: pd.DataFrame,
    start_date: str,
    end_date: str,
    out_dir: str = "outputs_prices",
):
    os.makedirs(out_dir, exist_ok=True)

    results = {}

    # Build contracts list
    contracts = [make_contract(row) for _, row in portfolio_df.iterrows()]

    # Qualify contracts (fills conId etc.)
    qualified = await ib.qualifyContractsAsync(*contracts)

    # Map back symbol -> qualified contract
    # Note: For Forex('EURUSD'), contract.symbol is usually 'EUR', localSymbol 'EUR.USD' etc.
    # so map using the portfolio symbols explicitly by iterating together.
    
    for (i, row) in enumerate(portfolio_df.itertuples(index=False)):
        sym = str(row.symbol).strip()
        contract = qualified[i]

        asset_class = str(row.asset_class).strip().upper()

        if asset_class in {"FX", "FOREX"}:
            whatToShow = "MIDPOINT"
            useRTH = False
        else:
            whatToShow = "TRADES"
            useRTH = True

        df = await fetch_daily_history(
            ib=ib,
            contract=contract,
            start_date=start_date,
            end_date=end_date,
            whatToShow=whatToShow,
            useRTH=useRTH,
        )

        results[sym] = df

        # Save
        out_path = os.path.join(out_dir, f"{sym}_{start_date}_{end_date}.csv".replace(":", "-"))
        df.to_csv(out_path, index=False)
        print(f"Saved {sym}: {len(df)} rows -> {out_path}")

    return results

In [23]:
### Run 

START_DATE = "1997-01-01"
END_DATE   = (pd.Timestamp.today().normalize() - pd.Timedelta(days=1)).strftime("%Y-%m-%d")
print("Using END_DATE =", END_DATE)

prices = await pull_all_assets_history(
    ib=ib,
    portfolio_df=portfolio,
    start_date=START_DATE,
    end_date=END_DATE,
)

Using END_DATE = 2026-02-06


Error 162, reqId 31: Historical Market Data Service error message:HMDS query returned no data: SPY@ARCA Trades, contract: Stock(conId=756733, symbol='SPY', exchange='ARCA', primaryExchange='ARCA', currency='USD', localSymbol='SPY', tradingClass='SPY')


Saved SPY: 5478 rows -> outputs_prices/SPY_1997-01-01_2026-02-06.csv


Error 162, reqId 57: Historical Market Data Service error message:HMDS query returned no data: TLT@ARCA Trades, contract: Stock(conId=15547841, symbol='TLT', exchange='ARCA', primaryExchange='NASDAQ', currency='USD', localSymbol='TLT', tradingClass='NMS')


Saved TLT: 5644 rows -> outputs_prices/TLT_1997-01-01_2026-02-06.csv


Error 162, reqId 80: Historical Market Data Service error message:HMDS query returned no data: GLD@ARCA Trades, contract: Stock(conId=51529211, symbol='GLD', exchange='ARCA', primaryExchange='ARCA', currency='USD', localSymbol='GLD', tradingClass='GLD')


Saved GLD: 5227 rows -> outputs_prices/GLD_1997-01-01_2026-02-06.csv


Error 162, reqId 102: Historical Market Data Service error message:HMDS query returned no data: EUR.USD@IDEALPRO Midpoint, contract: Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD')


Saved EURUSD: 5408 rows -> outputs_prices/EURUSD_1997-01-01_2026-02-06.csv


(1) IBKR returns Error 162 “no data” for a particular historical-data request

- usually occurs when one of the chunked requests has no available bars, commonly because the request goes beyond the instrument’s available history, hits a period before the product existed (e.g., GLD and TLT don’t have full history back to 2000), or the final chunk is outside IBKR’s coverage.
- Function treats this as a stopping condition (“no more data to fetch”), exits the loop, and then writes out whatever valid chunks were already collected.
  
(2) scripts are still successfully downloaded a large amount of data and saving it to a CSV.  

In [24]:
### for checking purpose 

prices["SPY"].tail()

,date,open,high,low,close,volume,average,barCount
5473,2026-02-02,689.60,696.93,689.43,695.41,13300172.0,694.791,128340
5474,2026-02-03,696.24,696.96,684.04,689.53,17311196.0,689.971,204107
5475,2026-02-04,690.40,691.45,681.76,686.19,17886889.0,686.758,202554
5476,2026-02-05,680.97,683.67,675.79,677.62,20992802.0,679.040,245383
5477,2026-02-06,681.43,692.31,680.85,690.62,13891402.0,687.247,145482


In [32]:
### paths 

SCENARIOS_PATH = "scenarios.csv"
PRICES_DIR = "outputs_prices"

### load inputs
scenarios = pd.read_csv(SCENARIOS_PATH)
scenarios.columns = [c.strip() for c in scenarios.columns]

portfolio, scenarios

(   symbol asset_class  exchange currency  weight
 0     SPY         ETF      ARCA      USD     0.4
 1     TLT         ETF      ARCA      USD     0.3
 2     GLD         ETF      ARCA      USD     0.1
 3  EURUSD          FX  IDEALPRO      USD     0.2,
                     scenario  start_date    end_date
 0                        afc  1997-07-01  1998-12-31
 1       russian_default_ltcm  1998-08-01  1998-10-31
 2                    dot_com  2000-03-01  2002-10-31
 3                        gfc  2007-08-01  2009-06-30
 4                flash_crash  2010-05-06  2010-05-07
 5         fukushima_meltdown  2011-03-01  2011-04-30
 6               sp_downgrade  2011-08-01  2011-08-31
 7           euro_debt_crisis  2014-08-01  2014-12-31
 8              taper_tantrum  2013-05-01  2013-09-30
 9             a50_turbulence  2015-06-01  2016-02-29
 10                    brexit  2016-06-01  2016-06-30
 11  us_presidential_election  2016-11-01  2016-11-30
 12               volmageddon  2018-02-01  2018

In [33]:
### load all price data 

price_data = {}

for sym in portfolio["symbol"]:
    file = [f for f in os.listdir(PRICES_DIR) if f.startswith(sym + "_")][0]
    df = pd.read_csv(os.path.join(PRICES_DIR, file))
    df["date"] = pd.to_datetime(df["date"])
    price_data[sym] = df.set_index("date").sort_index()

price_data.keys()

dict_keys(['SPY', 'TLT', 'GLD', 'EURUSD'])

In [34]:
### helper function to compute stress return (for one asset only) 
### it handles weekends / holidays and uses nearest trading day ≤ start/end (using the last available price before the date avoids look-ahead bias)

def stress_return_from_prices(px: pd.DataFrame, start_date: str, end_date: str):
    s = pd.Timestamp(start_date)
    e = pd.Timestamp(end_date)

    # nearest trading days ≤ start/end
    try:
        start_px = px.loc[:s].iloc[-1]["close"]
        end_px   = px.loc[:e].iloc[-1]["close"]
    except IndexError:
        return None  # not enough data

    return end_px / start_px - 1

In [37]:
### Compute asset-level stress returns
### produces one row per (scenario × asset

### Asset return = percentage price change over scenario window (independent of how large the position is in the portfolio)
### ie. (Price at end of scenario / Price at start of scenario) − 1

### Contribution = how many percentage points this asset added to or subtracted from the portfolio
### ie. weight × asset_return

asset_results = []

for _, sc in scenarios.iterrows():
    scenario = sc["scenario"]
    start = sc["start_date"]
    end = sc["end_date"]

    for _, row in portfolio.iterrows():
        sym = row["symbol"]
        weight = row["weight"]

        px = price_data[sym]
        r = stress_return_from_prices(px, start, end)

        asset_results.append({
            "scenario": scenario,
            "symbol": sym,
            "weight": weight,
            "asset_return": r,
            "contribution": None if r is None else weight * r
        })

asset_results = pd.DataFrame(asset_results)
asset_results[:30]

,scenario,symbol,weight,asset_return,contribution
0,afc,SPY,0.4,NaN,NaN
1,afc,TLT,0.3,NaN,NaN
2,afc,GLD,0.1,NaN,NaN
3,afc,EURUSD,0.2,NaN,NaN
4,russian_default_ltcm,SPY,0.4,NaN,NaN
5,russian_default_ltcm,TLT,0.3,NaN,NaN
6,russian_default_ltcm,GLD,0.1,NaN,NaN
7,russian_default_ltcm,EURUSD,0.2,NaN,NaN
8,dot_com,SPY,0.4,NaN,NaN
9,dot_com,TLT,0.3,NaN,NaN


The AFC (1997-1998), Russian Default & LTCM (1998) and Dotcom (2002-2002) rows are NaN because IBKR’s historical data feed only returned data starting from ~2004, so there is no price available on or before Dotcom scenario dates. 

To complete stress testing for the above scenarios, I will pull prices from yfinance for that specific window and compute the asset_return and contribution using the same formulas.

Only SPY is used because it is the only portfolio instrument with reliable and continuous price history covering that period. GLD and TLT were launched after Dotcom (so they do not have valid prices), and the EURUSD series can be inconsistent or unavailable for that exact window depending on the data provider. To avoid mixing incomplete or proxy series that could distort results, the stress tests are computed using SPY alone, ensuring the calculation is based on an instrument with verifiable data coverage and consistent methodology.

In [51]:
import pandas as pd
import yfinance as yf

### this function returns latest available trading price on or before given date (correct weekends/ holidays) 

def _nearest_leq(series: pd.Series, dt: pd.Timestamp):
    sub = series.loc[:dt]
    if sub.empty:
        return None
    return float(sub.iloc[-1])

### this function takes portfolio weights and a list of scenario windows (start/end dates), 
### downloads the relevant historical prices for all portfolio tickers from yfinance, 
### then computes for each (scenario, ticker) the asset return over that window and its contribution, 
### returning the results as a DataFrame.

def yfinance_stress_rows_for_scenarios(
    portfolio_df: pd.DataFrame,
    scenarios_df: pd.DataFrame,
    scenario_keys: list[str],
    price_col: str = "Close",
    buffer_days_before: int = 10,
    buffer_days_after: int = 2,
) -> pd.DataFrame:

    ### Extract scenario windows (start/end)
    windows: dict[str, tuple[pd.Timestamp, pd.Timestamp]] = {}
    scen_col = scenarios_df["scenario"].astype(str).str.lower()

    for key in scenario_keys:
        key_l = key.lower()
        hit = scenarios_df.loc[scen_col == key_l]
        if hit.empty:
            raise ValueError(f"Scenario '{key}' not found in scenarios_df['scenario'].")
        row = hit.iloc[0]
        windows[key_l] = (pd.Timestamp(row["start_date"]), pd.Timestamp(row["end_date"]))

    ### Prepare tickers + weights
    port = portfolio_df.copy()
    port["symbol"] = port["symbol"].astype(str).str.upper()
    port["weight"] = port["weight"].astype(float)
    tickers = port["symbol"].tolist()
    weights = dict(zip(port["symbol"], port["weight"]))

    ### Download once: from earliest start to latest end (with buffers)
    earliest_start = min(s for (s, e) in windows.values())
    latest_end = max(e for (s, e) in windows.values())

    dl_start = (earliest_start - pd.Timedelta(days=buffer_days_before)).strftime("%Y-%m-%d")
    dl_end   = (latest_end + pd.Timedelta(days=buffer_days_after)).strftime("%Y-%m-%d")

    df = yf.download(
        tickers=tickers,
        start=dl_start,
        end=dl_end,
        interval="1d",
        auto_adjust=True,
        progress=False,
        group_by="column",
    )

    if df.empty:
        raise ValueError("yfinance returned no data for the requested tickers/window.")

    ### Build {ticker: close_series} dict (handles one or many tickers)
    price_series: dict[str, pd.Series] = {}

    if isinstance(df.columns, pd.MultiIndex):
        if price_col not in df.columns.get_level_values(0):
            raise ValueError(f"Expected '{price_col}' in yfinance output columns.")
        closes = df[price_col].copy()  # DataFrame: index=date, cols=tickers
        for t in closes.columns:
            s = closes[t].dropna()
            s.index = pd.to_datetime(s.index)
            price_series[str(t).upper()] = s.sort_index()
    else:
        if price_col not in df.columns:
            raise ValueError(f"Expected '{price_col}' in yfinance output columns.")
        s = df[price_col].dropna()
        s.index = pd.to_datetime(s.index)
        price_series[tickers[0]] = s.sort_index()

    ### Build results rows
    rows = []
    for scen_key, (start, end) in windows.items():
        for sym in tickers:
            w = float(weights.get(sym, 0.0))
            series = price_series.get(sym)

            if series is None or series.empty:
                asset_return = np.nan
                contribution = np.nan
            else:
                start_px = _nearest_leq(series, start)
                end_px   = _nearest_leq(series, end)

                if start_px is None or end_px is None:
                    asset_return = np.nan
                    contribution = np.nan
                else:
                    asset_return = end_px / start_px - 1
                    contribution = w * asset_return

            rows.append({
                "scenario": scen_key,   # will be: afc / russian_default_ltcm / dot_com
                "symbol": sym,
                "weight": w,
                "asset_return": asset_return,
                "contribution": contribution
            })

    return pd.DataFrame(rows)


### computes YF rows for the three early scenarios then replace in asset_results 

early_scenarios = ["afc", "russian_default_ltcm", "dot_com"]

yf_early_rows = yfinance_stress_rows_for_scenarios(
    portfolio_df=portfolio,
    scenarios_df=scenarios,
    scenario_keys=early_scenarios
)

asset_results_updated = pd.concat(
    [
        asset_results[~asset_results["scenario"].astype(str).str.lower().isin(early_scenarios)],
        yf_early_rows
    ],
    ignore_index=True
).sort_values(["scenario", "symbol"]).reset_index(drop=True)

asset_results_updated[:20]



2 Failed downloads:
['GLD']: YFPricesMissingError('possibly delisted; no price data found  (1d 1997-06-21 -> 2002-11-02) (Yahoo error = "Data doesn\'t exist for startDate = 866865600, endDate = 1036213200")')
['EURUSD']: YFTzMissingError('possibly delisted; no timezone found')


,scenario,symbol,weight,asset_return,contribution
0,a50_turbulence,EURUSD,0.2,-0.004974,-0.000995
1,a50_turbulence,GLD,0.1,0.040702,0.004070
2,a50_turbulence,SPY,0.4,-0.085125,-0.034050
3,a50_turbulence,TLT,0.3,0.080244,0.024073
4,afc,EURUSD,0.2,NaN,NaN
5,afc,GLD,0.1,NaN,NaN
6,afc,SPY,0.4,0.408819,0.163528
7,afc,TLT,0.3,NaN,NaN
8,brexit,EURUSD,0.2,-0.007329,-0.001466
9,brexit,GLD,0.1,0.090823,0.009082


NaN values seen above are due to: 
- Yfinance does not reliably provide EURUSD data pre-2000 
- GLD ETF and TLT did not exist until 2004 and 2002 respectively.
  

In [54]:
asset_results_updated = asset_results_updated.dropna(
    subset=["asset_return", "contribution"]
).reset_index(drop=True)
asset_results_updated [:20]

,scenario,symbol,weight,asset_return,contribution
0,a50_turbulence,EURUSD,0.2,-0.004974,-0.000995
1,a50_turbulence,GLD,0.1,0.040702,0.004070
2,a50_turbulence,SPY,0.4,-0.085125,-0.034050
3,a50_turbulence,TLT,0.3,0.080244,0.024073
4,afc,SPY,0.4,0.408819,0.163528
5,brexit,EURUSD,0.2,-0.007329,-0.001466
6,brexit,GLD,0.1,0.090823,0.009082
7,brexit,SPY,0.4,-0.003805,-0.001522
8,brexit,TLT,0.3,0.064801,0.019440
9,carry_trade_unwind,EURUSD,0.2,0.023593,0.004719


In [55]:
### compute portfolio-level stress returns, which is the sum of all asset contributions in a scenario 
### this summarize how the entire portfolio would have performed during a historical crisis, rather than how individual assets moved in isolation. 


portfolio_stress_results = (
    asset_results_updated
    .groupby("scenario", as_index=False)
    .agg(
        portfolio_return=("contribution", "sum")
    )
)

portfolio_stress_results

,scenario,portfolio_return
0,a50_turbulence,-0.006901
1,afc,0.163528
2,brexit,0.025535
3,carry_trade_unwind,0.026102
4,covid19,0.151067
5,dot_com,-0.135386
6,euro_debt_crisis,0.028402
7,fitch_downgrade,-0.012217
8,flash_crash,-0.007708
9,fukushima_meltdown,0.046076


In [56]:
### rank scenarios by severity 

portfolio_stress_results = portfolio_stress_results.sort_values(
    by="portfolio_return"
)

portfolio_stress_results

,scenario,portfolio_return
5,dot_com,-0.135386
11,global_inflation,-0.093474
10,gfc,-0.079820
18,volmageddon,-0.026387
17,us_presidential_election,-0.024332
15,taper_tantrum,-0.021784
7,fitch_downgrade,-0.012217
8,flash_crash,-0.007708
0,a50_turbulence,-0.006901
13,russian_default_ltcm,-0.004984


The dot-com crash stands out as the most severe stress event, producing the largest negative portfolio return (−13.5%), followed by the global inflation shock and the Global Financial Crisis. These episodes reflect prolonged and systemic stress to equity markets, indicating that the portfolio is particularly vulnerable to sustained macroeconomic downturns rather than short-lived shocks. In contrast, events such as Volmageddon, the US presidential election, and the taper tantrum resulted in relatively mild losses, suggesting that volatility spikes and policy-related shocks had milder impact on overall portfolio performance.

But, it is important to note that for three early-period scenarios—AFC, the Russian default & LTCM episode, and the dot-com crash, SPY is the only ticker with available historical data, as other assets in the portfolio did not yet exist or lacked reliable price series. As a result, the computed portfolio returns for these scenarios effectively reflect pure equity exposure, without the stabilising influence of diversification from bonds, gold, or FX. This data constraint likely amplifies the magnitude of the observed portfolio movements, particularly for the dot-com crash, where the drawdown appears more severe due to the absence of offsetting asset classes. Consequently, these results should be interpreted as upper-bound stress outcomes rather than fully diversified portfolio responses. 

In [58]:
### identify main loss drivers per scenario 

top_contributors = (
    asset_results_updated
    .sort_values(["scenario", "contribution"])
    .groupby("scenario")
    .head(2)   # top 2 worst contributors per scenario
)

top_contributors

,scenario,symbol,weight,asset_return,contribution
2,a50_turbulence,SPY,0.4,-0.085125,-0.034050
0,a50_turbulence,EURUSD,0.2,-0.004974,-0.000995
4,afc,SPY,0.4,0.408819,0.163528
7,brexit,SPY,0.4,-0.003805,-0.001522
5,brexit,EURUSD,0.2,-0.007329,-0.001466
10,carry_trade_unwind,GLD,0.1,0.024450,0.002445
12,carry_trade_unwind,TLT,0.3,0.012373,0.003712
16,covid19,TLT,0.3,0.014931,0.004479
14,covid19,GLD,0.1,0.202049,0.020205
17,dot_com,SPY,0.4,-0.338466,-0.135386


The loss-driver breakdown shows that equities (SPY) are the dominant source of downside risk across the most severe stress scenarios, particularly during structurally equity-driven crises. 

In the dot-com crash and GFC, SPY alone accounts for the overwhelming share of losses, due to both its large portfolio weight and the depth of equity drawdowns in these periods. Similarly, in shorter market dislocations such as the flash crash, S&P downgrade, and Volmageddon, SPY remains the primary negative contributor, confirming that sudden volatility spikes transmit most strongly through equity exposure rather than through defensive assets.

In contrast, bond and commodity exposures emerge as the main loss drivers in macro-policy and rate-driven episodes. During the global inflation shock and the taper tantrum, TLT is the largest negative contributor, consistent with sharp increases in yields eroding long-duration bond prices. 

Gold (GLD) contributes negatively in several scenarios, such as the taper tantrum and euro debt crisis, indicating that gold does not always act as a hedge during liquidity-driven or dollar-strength episodes. 

Currency exposure (EURUSD) tends to play a secondary but non-negligible role, particularly during European-centric crises and volatility events, where FX movements exacerbate losses rather than offset them. 

Overall, equity risk dominates during market crashes, while interest-rate risk becomes the primary loss driver during inflationary and policy-tightening regimes. 

Simple glossary: 

1. afc = The Asian Financial Crisis (1997–1998), triggered by currency devaluations and capital flight across East and Southeast Asia, leading to sharp equity market declines and regional financial instability.
2. russian_default_ltcm = Russia’s 1998 sovereign debt default combined with the collapse of Long-Term Capital Management, which caused severe stress in global bond markets and highlighted systemic leverage risk.
3. dot_com = The bursting of the late-1990s technology bubble, marked by a prolonged collapse in equity valuations following excessive speculation in internet and growth stocks.
4. gfc = The Global Financial Crisis (2007–2009), driven by the US subprime mortgage collapse and failures in the global banking system, resulting in a deep and broad-based market downturn.
5. flash_crash = The May 2010 Flash Crash, a sudden and extreme intraday equity market sell-off caused by liquidity evaporation and automated trading feedback loops.
6. fukushima_meltdown = The 2011 Fukushima nuclear disaster following a major earthquake and tsunami in Japan, which disrupted global supply chains and triggered risk-off market behaviour.
7. sp_downgrade = The 2011 downgrade of US sovereign credit by Standard & Poor’s, which increased market uncertainty despite US Treasuries retaining safe-haven status.
8. euro_debt_crisis = The European sovereign debt crisis, characterised by concerns over fiscal sustainability in countries such as Greece, Spain, and Italy, leading to stress in European assets and currencies.
9. taper_tantrum = The 2013 market sell-off following the US Federal Reserve’s signal of tapering quantitative easing, which caused a sharp rise in bond yields and losses in duration-sensitive assets.
10. a50_turbulence = The 2015–2016 turbulence in Chinese equity markets, including sharp declines in the China A-share market and heightened volatility linked to growth concerns and policy uncertainty.
11. brexit = The 2016 UK referendum decision to leave the European Union, which led to immediate currency depreciation and heightened volatility across European financial markets.
12. us_presidential_election = The 2016 US presidential election, which generated short-term market volatility driven by policy uncertainty and shifts in investor expectations.
13. volmageddon = The February 2018 volatility spike, during which inverse volatility products collapsed and equity markets experienced abrupt losses.
14. covid19 = The global COVID-19 pandemic, which caused an abrupt economic shutdown and extreme market volatility, followed by strong policy-driven asset price recoveries.
15. global_inflation = The post-pandemic global inflation surge, characterised by rising interest rates and tightening monetary policy, negatively impacting bonds and growth-oriented assets.
16. fitch_downgrade = Fitch’s 2023 downgrade of US sovereign credit, reflecting concerns over fiscal governance and contributing to short-term market volatility.
17. carry_trade_unwind = A rapid unwinding of global carry trades, typically driven by rising volatility or funding currency appreciation, leading to losses in leveraged FX positions.
18. trump_tariffs = The escalation of US–China trade tensions through tariff announcements under the Trump administration, increasing uncertainty for global trade and risk assets.
19. gold_silver_bust = A sharp correction in precious metals prices following a speculative run-up, reflecting shifts in inflation expectations, real yields, or investor positioning.